# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [ ]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [ ]:
from agentics import AG
from crewai.tools import tool
from ddgs import DDGS


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))

## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [ ]:
from crewai import Agent

doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=AG.get_llm_provider(),
)

## Define a Task

In [ ]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True,
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [ ]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [ ]:
result = crew.kickoff(inputs={"question": input("Ask your question> ")})

## Display the results as markdown

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.raw))